# Implicit Trajectory combined with RRT #
- Work space: 3D with obstacles
- Obstacle type: static and known
- Agent Model: Dubins airplane (Owen2015)
- Integration Algorithm: 4th Order Runge Kutta
- Implicit Curve: Time-invariant Circle
- Distance Metric: admissionable heuristic, based on time-optimal dubins-airplane paths (altitude difference is integrated in a linear fashion)

In [9]:
# load the code
%run 'MAIN_CODE.ipynb'
import random
random.seed( 99 ) #arbitrary

# Performance Experiments

## 1st stage: VF-RRT+impl.Curve vs. RRT+goal bias

In [10]:
import time

#Parameters in General
NumOfExperiments = 10
qstart = np.array([5.,5.,2.,np.pi])# specific initial point
MaxNodes = 300
MAXiter = MaxNodes*3

# Experiment parameters for VF-RRT
lambdaNsteps=40
Es=0.4
lamdb=1.0

# Experiment parameters for naive RRT
goalBias=0.25

In [11]:
#specific obstacles, fixed, known, not moving
# reset the RNG
random.seed( 500 ) #arbitrary
Obstacles = CreateSphericalObstacles(N=100,r=2.)
while isBlocked(qstart, Obstacles):
    Obstacles = CreateSphericalObstacles(N=100,r=2.)

In [12]:
# initilize
VFRRTnodes = list()
VFRRTupstream = list()
VFRRTnodes = list()
VFRRTexetime = list()
VFRRTwork = list()
VFRRTsuccess = list()
VFRRTiter = list()
VFRRTeuclLength = list()
VFRRTanglLength = list()
    
RRTnodes = list()
RRTupstream = list()
RRTnodes = list()
RRTexetime = list()
RRTwork = list()
RRTsuccess = list()
RRTiter = list()
RRTeuclLength = list()
RRTanglLength = list()

print('#\tt\tsuccess\twork\tupstr\tnodes\titer\teuDist\tangDist\thiLim\tloLim')
print('VFRRT')
for IofExp in range(0,NumOfExperiments):
    # Execute and return Tree
    stampBegin = time.process_time()
    T = VFRRT(qstart,VF,lambdaNsteps,Es,lamdb,MaxNodes,MAXiter,Obstacles,bounds=bounds)
    CPUtimePassed = time.process_time()-stampBegin

    
    if T.successful:
        VFRRTexetime.append(CPUtimePassed)
        VFRRTiter.append(T.iterations)
        VFRRTsuccess.append(IofExp)
        IDpath,Datapath,Edgepath = T.get_solution()
        work,upstream = T.get_WorkAndUpstreamOfPath(Datapath)
        euclDist,anglDist = T.get_length(Datapath)
        VFRRTnodes.append(T.NumNodes)
        VFRRTupstream.append(upstream)
        VFRRTwork.append(work)
        VFRRTeuclLength.append(euclDist)
        VFRRTanglLength.append(anglDist)
        
        print('%d\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\t%s\t%s' % (IofExp,CPUtimePassed,work,upstream,T.NumNodes,T.iterations,euclDist,anglDist,str(T.lambd_high),str(T.lambd_low)))
    else:
        print('%d\t%.3f\tFalse\t\t\t%d\t%d\t\t\t%s\t%s' % (IofExp,CPUtimePassed,T.NumNodes,T.iterations,str(T.lambd_high),str(T.lambd_low)))
    
    #T.plot(DrawTrajectory=True,DrawCollisionNodes=True)
    #T.plot(obs=Obstacles,OnlyClosestObs=True,DrawCollisionNodes=True)
    #DrawStats(T)

print('\nStats for successful runs only\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmin' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.min(VFRRTexetime),
                                                                np.min(VFRRTwork),
                                                                np.min(VFRRTupstream),
                                                                np.min(VFRRTnodes),
                                                                np.min(VFRRTiter),
                                                                np.min(VFRRTeuclLength),
                                                                np.min(VFRRTanglLength)))
print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmean' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.mean(VFRRTexetime),
                                                                np.mean(VFRRTwork),
                                                                np.mean(VFRRTupstream),
                                                                np.mean(VFRRTnodes),
                                                                np.mean(VFRRTiter),
                                                                np.mean(VFRRTeuclLength),
                                                                np.mean(VFRRTanglLength)))
print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmax' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.max(VFRRTexetime),
                                                                np.max(VFRRTwork),
                                                                np.max(VFRRTupstream),
                                                                np.max(VFRRTnodes),
                                                                np.max(VFRRTiter),
                                                                np.max(VFRRTeuclLength),
                                                                np.max(VFRRTanglLength)))
print('\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tstd\n' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.std(VFRRTexetime),
                                                                np.std(VFRRTwork),
                                                                np.std(VFRRTupstream),
                                                                np.std(VFRRTnodes),
                                                                np.std(VFRRTiter),
                                                                np.std(VFRRTeuclLength),
                                                                np.std(VFRRTanglLength)))
print('RRT')
for IofExp in range(0,NumOfExperiments):
    gt = np.mod(IofExp/NumOfExperiments*2*np.pi+np.pi/2,2*np.pi)
    qgoal=(
        xc+r*np.cos(2*np.pi*IofExp/NumOfExperiments),
        yc+r*np.sin(2*np.pi*IofExp/NumOfExperiments),
        zc,
        gt
    )
    stampBegin = time.process_time()
    #pdb.set_trace()
    T = RRT(qstart,qgoal,MaxNodes,MAXiter,Obstacles,bounds,goalBias)
    CPUtimePassed = time.process_time()-stampBegin

    
    if T.successful:
        RRTexetime.append(CPUtimePassed)
        RRTiter.append(T.iterations)
        T.VF = VF
        RRTsuccess.append(IofExp)
        IDpath,Datapath,Edgepath = T.get_solution()
        work,upstream = T.get_WorkAndUpstreamOfPath(Datapath)
        euclDist,anglDist = T.get_length(Datapath)
        RRTnodes.append(T.NumNodes)
        RRTupstream.append(upstream)
        RRTwork.append(work)
        RRTeuclLength.append(euclDist)
        RRTanglLength.append(anglDist)
        
        print('%d\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f' % (IofExp,CPUtimePassed,work,upstream,T.NumNodes,T.iterations,euclDist,anglDist))
    else:
        print('%d\t%.3f\tFalse\t\t\t%d\t%d\t\t' % (IofExp,CPUtimePassed,T.NumNodes,T.iterations))
    #T.plot(obs=Obstacles,OnlyClosestObs=True,DrawCollisionNodes=True)
#plt.show()
if len(RRTsuccess):
    print('\nStats for successful runs only\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmin' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.min(RRTexetime),
                                                                    np.min(RRTwork),
                                                                    np.min(RRTupstream),
                                                                    np.min(RRTnodes),
                                                                    np.min(RRTiter),
                                                                    np.min(RRTeuclLength),
                                                                    np.min(RRTanglLength)))

    print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmean' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.mean(RRTexetime),
                                                                    np.mean(RRTwork),
                                                                    np.mean(RRTupstream),
                                                                    np.mean(RRTnodes),
                                                                    np.mean(RRTiter),
                                                                    np.mean(RRTeuclLength),
                                                                    np.mean(RRTanglLength)))
    print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmax' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.max(RRTexetime),
                                                                    np.max(RRTwork),
                                                                    np.max(RRTupstream),
                                                                    np.max(RRTnodes),
                                                                    np.max(RRTiter),
                                                                    np.max(RRTeuclLength),
                                                                    np.max(RRTanglLength)))
    print('\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tstd\n' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.std(RRTexetime),
                                                                    np.std(RRTwork),
                                                                    np.std(RRTupstream),
                                                                    np.std(RRTnodes),
                                                                    np.std(RRTiter),
                                                                    np.std(RRTeuclLength),
                                                                    np.std(RRTanglLength)))



#	t	success	work	upstr	nodes	iter	euDist	angDist	hiLim	loLim
VFRRT
0	0.458	True	4.720	1.188	79	79	4.849	13.217	False	False
1	0.334	True	4.515	1.781	75	75	4.706	13.397	False	False
2	0.341	True	4.591	2.042	79	79	4.807	15.929	False	False
3	0.302	True	4.719	2.636	72	72	5.001	14.722	False	False
4	0.477	True	4.638	1.423	87	87	4.790	14.255	False	False
5	0.445	True	4.497	1.960	76	76	4.710	10.952	False	False
6	0.573	True	5.068	4.184	101	101	5.499	16.245	False	False
7	0.375	True	4.721	1.871	68	68	4.922	17.183	False	False
8	0.363	True	4.390	6.726	70	70	5.103	17.122	False	False
9	0.345	True	4.627	1.753	78	78	4.814	12.187	False	False

Stats for successful runs only
100%	0.302	True	4.390	1.188	68	68	4.706	10.952	min
100%	0.401	True	4.648	2.556	78	78	4.920	14.521	mean
100%	0.573	True	5.068	6.726	101	101	5.499	17.183	max

100%	0.080	True	0.174	1.598	9	9	0.226	2.002	std

RRT
0	1.522	False			300	300		
1	1.521	False			300	313		
2	1.634	False			300	301		
3	2.510	False			300	403		
4	2.078	False			300	346		